In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import numpy as np
from scipy.stats import chi2_contingency

In [ ]:
clean_df = pd.read_csv("../data/cleaned/clean_data.csv")
clean_df

## Create CSV for tables that does not have Foreign Keys

In [ ]:
"""
1. City: city_id, name
2. meals: meal_id, meal_type
3. review_summary: review_id, avg_rating, total_reviews_count, excellent, terrible, restaurant_id
4. dietary_options: diet_id, vegetarian_friendly, vegan_options, gluten_free
5. price_range: price_id, price_range
6. language: language_id, name
"""

#### CITY TABLE

In [ ]:
city = pd.DataFrame({"city_id": [i+1 for i in range(clean_df["city"].nunique())], "city_name": clean_df["city"].unique()})
city

In [ ]:
city.to_csv("city.csv", index=False, sep=";", encoding="utf-8")

#### MEALS TABLE

In [ ]:
meal_types_df = clean_df["meals"].str.split(",", expand=True)

meal_types_list = []
for row in range(len(meal_types_df)):
    for col in range(5):
        if meal_types_df.iloc[row, col] and meal_types_df.iloc[row, col] not in meal_types_list:
            meal_types_list.append(meal_types_df.iloc[row, col])

In [ ]:
meal_types_list = [ elem.strip() for elem in meal_types_list ]
meal_types_set = set(meal_types_list)
print(meal_types_set)

In [ ]:
meals = pd.DataFrame({"meal_id": [i+1 for i in range(len(list(meal_types_set)))], "meal_type": list(meal_types_set)})
meals

In [ ]:
meals.to_csv("meals.csv", index=False, sep=";", encoding="utf-8")

#### DIETARY OPTIONS TABLE

In [ ]:
dietary_options1 = pd.DataFrame({"diet_id": [i+1 for i in range(clean_df["vegetarian_friendly"].nunique())  ], 
                                "vegetarian_friendly": clean_df["vegetarian_friendly"].unique(), 
                                "vegan_options": clean_df["vegan_options"].unique(), 
                                "gluten_free": clean_df["gluten_free"].unique()})

data = {"diet_id": 3, "vegetarian_friendly": "y", "vegan_options": "Y", "gluten_free": "N"}
data1 = {"diet_id": 4, "vegetarian_friendly": "n", "vegan_options": "N", "gluten_free": "Y"}
data2 = {"diet_id": 5, "vegetarian_friendly": "y", "vegan_options": "N", "gluten_free": "Y"}

dietary_options2 = pd.DataFrame([data, data1, data2])

dietary_options = pd.concat([dietary_options1, dietary_options2], axis=0)
dietary_options.reset_index(drop = True, inplace=True)
dietary_options

In [ ]:
dietary_options.to_csv("dietary_options.csv", index=False, sep=";", encoding="utf-8")

#### PRICE RANGE TABLE

In [ ]:
price = pd.DataFrame({"price_id": [i+1 for i in range(clean_df["price_range"].nunique())], 
                                "price_range": clean_df["price_range"].unique()})
price

In [ ]:
price.to_csv("price.csv", index=False, sep=";", encoding="utf-8")

#### LANGUAGE TABLE

In [ ]:
language = pd.DataFrame({"language_id": [i+1 for i in range(clean_df["default_language"].nunique())], 
                                "language_name": clean_df["default_language"].unique()})
language

In [ ]:
language.to_csv("language.csv", index=False, sep=";", encoding="utf-8")

#### REVIEW SUMMARY TABLE

In [ ]:
review_summary = (
    clean_df[[
        "restaurant_name",
        "city",
        "avg_rating",
        "total_reviews_count",
        "excellent",
        "terrible"
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)

review_summary.insert(0, "review_id", review_summary.index + 1)
review_summary

In [ ]:
review_summary.to_csv("review_summary.csv", index=False, sep=";", encoding="utf-8")

## Create CSV for tables that does have Foreign Keys

In [ ]:
"""
1. country: country_id, name, city_id

2. location: location_id, lat, long, city_id

3. restaurant_language: id, rest_id, lang_id

4. restaurant_meals: id, rest_id, meal_id

5. restaurant_dietary_options: id, rest_id, diet_id
"""

#### COUNTRY TABLE

In [ ]:
city_table = (
    clean_df[["city", "country"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

city_table["city_id"] = city_table.index + 1

country = (
    city_table[["city_id", "country"]]
    .copy()
    .reset_index(drop=True)
)

country["country_id"] = country.index + 1
country.rename(columns={"country": "country_name"}, inplace=True)

country = country[["country_id", "city_id", "country_name"]]
country

In [ ]:
country.to_csv("country.csv", index=False, sep=";", encoding="utf-8")

#### LOCATION TABLE

In [ ]:
city_location = (
    clean_df[["restaurant_name", "city", "latitude", "longitude"]]
    .drop_duplicates()
)

location = (
    city_location
    .merge(city, left_on="city", right_on = "city_name",how="left")
)
location = location.reset_index(drop=True)
location.insert(0, "location_id", location.index + 1)

location = location[["location_id", "city_id", "latitude", "longitude"]]

location

In [ ]:
location.to_csv("location.csv", index=False, sep=";", encoding="utf-8")

#### RESTAURANT LANGUAGE TABLE

In [ ]:
restaurant_language = (
    clean_df[[
        "restaurant_name", "default_language"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

#display(restaurant.head())
#display(language.head())

restaurant_language = restaurant_language.merge(
    language[["language_id", "language_name"]],
    left_on="default_language", right_on = "language_name",
    how="inner"
)


restaurant_language = restaurant_language.merge(
    restaurant[["restaurant_id", "restaurant_name"]],
    on="restaurant_name",
    how="inner"
)

restaurant_language = restaurant_language.reset_index(drop=True)

restaurant_language.insert(0, "id", restaurant_language.index + 1)

restaurant_language = restaurant_language[["id", "language_id", "restaurant_id"]]

display(restaurant_language.head())

In [ ]:
restaurant.shape

In [ ]:
restaurant_language.to_csv("restaurant_language.csv", index=False, sep=";", encoding="utf-8"))

#### RESTAURANT MEALS TABLE

In [ ]:
restaurant_meals = (
    clean_df[[
        "restaurant_name", "meals"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

restaurant_meals = restaurant_meals.merge(
    meals[["meal_id", "meal_type"]],
    left_on="meals", right_on = "meal_type", # type doesn't match
    how="inner"
)


restaurant_meals = restaurant_meals.merge(
    restaurant[["restaurant_id", "restaurant_name"]],
    on="restaurant_name",
    how="inner"
)


restaurant_meals = restaurant_meals.reset_index(drop=True)

restaurant_meals.insert(0, "id", restaurant_meals.index + 1)

restaurant_meals = restaurant_meals[["id", "meal_id", "restaurant_id"]]

restaurant_meals.head()

In [ ]:
restaurant_meals.to_csv("restaurant_meals.csv", index=False, sep=";", encoding="utf-8"))

#### RESTAURANT DIETARY OPTIONS TABLE

In [ ]:
restaurant_dietary_options = (
    clean_df[[
        "restaurant_name", "vegetarian_friendly"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dietary_options = dietary_options.drop_duplicates(
    subset=["vegetarian_friendly"]
)

restaurant_dietary_options = restaurant_dietary_options.merge(
    dietary_options[["diet_id", "vegetarian_friendly"]],
    on = "vegetarian_friendly",
    how="left"
)


restaurant_dietary_options = restaurant_dietary_options.merge(
    restaurant[["restaurant_id", "restaurant_name"]],
    on="restaurant_name",
    how="left"
)

restaurant_dietary_options = restaurant_dietary_options.reset_index(drop=True)
restaurant_dietary_options.insert(0, "id", restaurant_dietary_options.index + 1)

restaurant_dietary_options = restaurant_dietary_options[["id", "diet_id", "restaurant_id"]]

display(restaurant_dietary_options.head())

In [ ]:
restaurant_dietary_options.to_csv("restaurant_dietary_options.csv", index=False, sep=";", encoding="utf-8"))

## Create CSV for the restaurant table

In [ ]:
"""
Restaurant: restaurant_id, name, claimed, awards, open_days_per_week, original_open_hours, location_id, price_id, review_id
"""

In [ ]:
restaurant = (
    clean_df[[
        "restaurant_name",
        "claimed",
        "awards",
        "open_days_per_week",
        "original_open_hours",
        "city",              # needed for location merge
        "price_range",       # needed for price merge
        "avg_rating",
        "total_reviews_count",
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)

restaurant.insert(0, "restaurant_id", restaurant.index + 1)

# restaurant.rename(columns={"restaurant_name": "name"}, inplace=True)

restaurant = restaurant.merge(
    location,
    on=["restaurant_name","city"],
    how="inner"
)

restaurant = restaurant.merge(
    price[["price_id", "price_range"]],
    on="price_range",
    how="inner"
)

restaurant = restaurant.merge(
    review_summary,
    on=["restaurant_name","city"],
    how="inner"
)

restaurant = restaurant.drop(columns=['avg_rating_y', 'total_reviews_count_y', "latitude", "longitude"])

restaurant = restaurant[[
    "restaurant_id",
    "restaurant_name",
    "claimed",
    "awards",
    "open_days_per_week",
    "original_open_hours",
    "location_id",
    "price_id",
    "review_id"
]]

display(restaurant.head())

In [ ]:
restaurant.to_csv("restaurant.csv", index=False, sep=";", encoding="utf-8")